# ONNX 测试样例

In [1]:
%cd ..
from utils.onnx_utils import (
    get_input_data_shape_dict,
    make_constant_node, get_onnxruntime_output,
    get_tvm_output, get_tvm_output_with_vm,
    verify_with_ort, verify_with_ort_with_inputs,
    quantize_and_verify_with_ort
)

/media/pc/data/lxw/ai/tvm/xinetzone/tvm-book/doc/tutorials/frontend


In [2]:
# import torch
# import torchvision
# from torch import nn
# # from torch.nn import Linear, Module, Sequential

In [3]:
import numpy as np
import onnx
from onnx import TensorProto, helper, numpy_helper
import tvm

## `reshape`

In [4]:
in_shape = (4, 3, 3, 4)
ref_shape = (6, 2, 4, 3)

ref_array = np.array(ref_shape)
ref_node = onnx.helper.make_node(
    "Constant",
    inputs=[],
    outputs=["ref_in"],
    value=onnx.helper.make_tensor(
        name="const_tensor",
        data_type=onnx.TensorProto.INT32,
        dims=ref_array.shape,
        vals=ref_array.flatten().astype(int),
    ),
)
reshape_node = helper.make_node("Reshape", ["in", "ref_in"], ["out"])

graph = helper.make_graph(
    [ref_node, reshape_node],
    "reshape_test",
    inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
    outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(ref_shape))],
)

model = helper.make_model(graph, producer_name="reshape_test")

In [5]:
target = "llvm"
dev = tvm.cpu()
x = np.random.uniform(size=in_shape).astype("int32")
tvm_out = get_tvm_output(model, x, target, dev, ref_shape, "float32")
np.testing.assert_allclose(ref_shape, tvm_out.shape)

## `double_reshape`

In [6]:
in_shape = (4, 3, 3, 4)
ref_shape = (6, 2, 4, 3)

ref_array = np.array(ref_shape)
ref_node = onnx.helper.make_node(
    "Constant",
    inputs=[],
    outputs=["ref_in"],
    value=onnx.helper.make_tensor(
        name="const_tensor",
        data_type=onnx.TensorProto.INT32,
        dims=ref_array.shape,
        vals=ref_array.flatten().astype(int),
    ),
)
reshape_node1 = helper.make_node("Reshape", ["in", "ref_in"], ["out1"])
reshape_node2 = helper.make_node("Reshape", ["in", "ref_in"], ["out2"])
add_node = helper.make_node("Add", ["out1", "out2"], ["out"])

graph = helper.make_graph(
    [ref_node, reshape_node1, reshape_node2, add_node],
    "reshape_test",
    inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
    outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(ref_shape))],
)

model = helper.make_model(graph, producer_name="reshape_test")

x = np.random.uniform(size=in_shape).astype("int32")
tvm_out = get_tvm_output(model, x, target, dev, ref_shape, "float32")
np.testing.assert_allclose(ref_shape, tvm_out.shape)

## `expand`

In [7]:
def _test_expand(name, data, shape, ref_data, dtype="int32"):
    shape_array = np.array(shape)
    if dtype == "int32":
        shape_node = onnx.helper.make_node(
            "Constant",
            inputs=[],
            outputs=["shape"],
            value=onnx.helper.make_tensor(
                name="const_tensor",
                data_type=onnx.TensorProto.INT32,
                dims=shape_array.shape,
                vals=shape_array.flatten().astype("int32"),
            ),
        )
    elif dtype == "int64":
        shape_node = onnx.helper.make_node(
            "Constant",
            inputs=[],
            outputs=["shape"],
            value=onnx.helper.make_tensor(
                name="const_tensor",
                data_type=onnx.TensorProto.INT64,
                dims=shape_array.shape,
                vals=shape_array.flatten().astype("int64"),
            ),
        )
    else:
        raise TypeError("Invalid dtype")
    expand_node = helper.make_node("Expand", ["in", "shape"], ["out"])

    graph = helper.make_graph(
        [shape_node, expand_node],
        "expand_test",
        inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(data.shape))],
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(ref_data.shape))],
    )

    model = helper.make_model(graph, producer_name=name)

    tvm_out = get_tvm_output_with_vm(model, data, target, dev, freeze_params=True)
    np.testing.assert_allclose(ref_data, tvm_out)

in_shape = (3, 1)
shape = (3, 4)
data = np.random.uniform(size=in_shape).astype(np.float32)
ref_data = np.tile(data, 4)
_test_expand("expand_with_dim_unchanged_test", data, shape, ref_data, "int32")
_test_expand("expand_with_dim_unchanged_test", data, shape, ref_data, "int64")

in_shape = (3, 1)
shape = (2, 1, 6)
data = np.random.uniform(size=in_shape).astype(np.float32)
ref_data = data * np.ones(shape, dtype=np.float32)
_test_expand("expand_larger_target_shape_test", data, shape, ref_data, "int32")
_test_expand("expand_larger_target_shape_test", data, shape, ref_data, "int64")

in_shape = (1, 1)
shape = (3,)
data = np.random.uniform(size=in_shape).astype(np.float32)
ref_data = data * np.ones(shape, dtype=np.float32)
_test_expand("expand_smaller_target_shape_test", data, shape, ref_data, "int32")
_test_expand("expand_smaller_target_shape_test", data, shape, ref_data, "int64")

## `depth_to_space`

In [8]:
def verify_depth_to_space(inshape, outshape, mode, block_size):
    node = onnx.helper.make_node(
        "DepthToSpace", inputs=["x"], outputs=["y"], blocksize=block_size
    )

    graph = helper.make_graph(
        [node],
        "depth_to_space_test",
        inputs=[helper.make_tensor_value_info("x", TensorProto.FLOAT, list(inshape))],
        outputs=[helper.make_tensor_value_info("y", TensorProto.FLOAT, list(outshape))],
    )

    model = helper.make_model(graph, producer_name="depth_to_space_test")

    verify_with_ort(model, [inshape], [outshape], target, dev)

# current onnx.checker use OpSet-1 version of DepthToSpace, which doesn't have a mode argument.
# TO-DO, we can add mode argument to test CRD mode and DCR mode
# in the future when we update to a newer onnx version.
verify_depth_to_space((1, 8, 2, 3), (1, 2, 4, 6), mode="CRD", block_size=2)

## `space_to_depth`

In [9]:
def verify_space_to_depth(inshape, outshape, block_size):
    node = onnx.helper.make_node(
        "SpaceToDepth", inputs=["x"], outputs=["y"], blocksize=block_size
    )

    graph = helper.make_graph(
        [node],
        "space_to_depth_test",
        inputs=[helper.make_tensor_value_info("x", TensorProto.FLOAT, list(inshape))],
        outputs=[helper.make_tensor_value_info("y", TensorProto.FLOAT, list(outshape))],
    )

    model = helper.make_model(graph, producer_name="space_to_depth_test")

    verify_with_ort(model, [inshape], [outshape], target, dev)

verify_space_to_depth((1, 1, 4, 6), (1, 4, 2, 3), 2)

## `shape`

In [10]:
in_shape = (4, 3, 3, 4)
ref_shape = (6, 2, 4, 3)

ref_array = np.array(ref_shape)
ref_node = onnx.helper.make_node(
    "Constant",
    inputs=[],
    outputs=["ref_in"],
    value=onnx.helper.make_tensor(
        name="const_tensor",
        data_type=onnx.TensorProto.INT32,
        dims=ref_array.shape,
        vals=ref_array.flatten().astype(int),
    ),
)
reshape_node = helper.make_node("Reshape", ["in", "ref_in"], ["out"])

shape_node = helper.make_node("Shape", ["out"], ["final_out"])

graph = helper.make_graph(
    [ref_node, reshape_node, shape_node],
    "shape_test",
    inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
    outputs=[helper.make_tensor_value_info("final_out", TensorProto.FLOAT, list(ref_shape))],
)

model = helper.make_model(graph, producer_name="shape_test")

x = np.random.uniform(size=in_shape).astype("int32")
tvm_out = get_tvm_output(model, x, target, dev, ref_shape, "int32")
np.testing.assert_allclose(ref_shape, tvm_out)

## `power`

In [11]:
def _test_power_iteration(x_shape, y_shape):
    if isinstance(y_shape, int):
        y_shape = [y_shape]

    x = np.random.uniform(size=x_shape).astype(np.float32)
    y = np.random.uniform(size=y_shape).astype(np.float32)

    np_res = np.power(x, y).astype(np.float32)

    res = helper.make_node("Pow", ["x", "y"], ["out"])

    graph = helper.make_graph(
        [res],
        "power_test",
        inputs=[
            helper.make_tensor_value_info("x", TensorProto.FLOAT, list(x_shape)),
            helper.make_tensor_value_info("y", TensorProto.FLOAT, list(y_shape)),
        ],
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(np_res.shape))],
    )

    model = helper.make_model(graph, producer_name="power_test")

    tvm_out = get_tvm_output(model, [x, y], target, dev, np_res.shape)
    tvm.testing.assert_allclose(np_res, tvm_out, rtol=1e-5, atol=1e-5)

_test_power_iteration((1, 3), (1))
_test_power_iteration((2, 3), (2, 3))
_test_power_iteration((2, 3), (1, 3))

## `range`

In [13]:
def verify_range(start, limit, delta, dtype):
    dtype_map = {
        "float32": TensorProto.FLOAT,
        "int32": TensorProto.INT32,
        "int64": TensorProto.INT64,
    }
    dtype_onnx = dtype_map[dtype]
    y = helper.make_node("Range", ["start", "limit", "delta"], ["output"])
    graph = helper.make_graph(
        [y],
        "range_test",
        inputs=[
            helper.make_tensor_value_info("start", dtype_onnx, []),
            helper.make_tensor_value_info("limit", dtype_onnx, []),
            helper.make_tensor_value_info("delta", dtype_onnx, []),
        ],
        outputs=[
            helper.make_tensor_value_info(
                "output", dtype_onnx, np.arange(start, limit, delta).shape
            )
        ],
    )
    model = helper.make_model(graph, producer_name="range_test")
    inputs = [np.array(x).astype(dtype) for x in [start, limit, delta]]
    verify_with_ort_with_inputs(model, inputs, target=target, dev=dev, use_vm=True)

for t in ["float32", "int32", "int64"]:
    verify_range(0, 10, 1, t)
    verify_range(2, 8, 2, t)
    verify_range(-3, 6, 4, t)
    verify_range(-2, -7, -1, t)

## `squeeze`

In [14]:
def test_squeeze_once(in_shape, out_shape, axes=None):
    y = helper.make_node("Squeeze", ["in"], ["out"], axes=axes)

    graph = helper.make_graph(
        [y],
        "squeeze_test",
        inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(out_shape))],
    )

    model = helper.make_model(graph, producer_name="squeeze_test")
    x = np.random.uniform(size=in_shape).astype("float32")
    verify_with_ort_with_inputs(model, [x], [out_shape], target=target, dev=dev, opset=11)

test_squeeze_once((1, 3, 1, 3, 1, 1), (3, 3), [0, 2, 4, 5])
test_squeeze_once((1, 3, 1, 3, 1, 1), (3, 3))  # empty axis.
test_squeeze_once((), ())  # scalar testing.

## `flatten`

In [15]:
def verify_flatten(in_shape, axis, ref_shape):
    flatten_node = helper.make_node("Flatten", ["in"], ["out"], axis=axis)

    graph = helper.make_graph(
        [flatten_node],
        "flatten_test",
        inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(ref_shape))],
    )

    model = helper.make_model(graph, producer_name="flatten_test")
    verify_with_ort(model, [in_shape], target=target, dev=dev)

verify_flatten((1, 3, 4, 4), 1, (1, 48))
verify_flatten((1,), 1, (1, 1))


## `unsqueeze`

In [16]:
in_shape = (3, 3)
axis = (0, 3, 4)
out_shape = (1, 3, 3, 1, 1)
y = helper.make_node("Unsqueeze", ["in"], ["out"], axes=list(axis))

graph = helper.make_graph(
    [y],
    "squeeze_test",
    inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
    outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(out_shape))],
)

model = helper.make_model(graph, producer_name="squeeze_test")
verify_with_ort(model, [in_shape], target=target, dev=dev, opset=11)


## `unsqueeze_with_neg_axes`

In [17]:
def verify_unsqueeze_with_neg_axes(opset=11):
    in_shape = (2, 3, 4)
    axis = (-2, -1)
    out_shape = (2, 3, 4, 1, 1)
    if opset < 13:
        y = helper.make_node("Unsqueeze", ["in"], ["out"], axes=list(axis))
        nodes = [y]
    else:
        axes = np.array(list(axis)).astype(np.int64)
        axes = helper.make_node(
            "Constant",
            inputs=[],
            outputs=["axes"],
            value=onnx.helper.make_tensor(
                name="const_axes",
                data_type=onnx.TensorProto.INT64,
                dims=axes.shape,
                vals=axes.flatten().astype(int),
            ),
        )
        y = helper.make_node("Unsqueeze", ["in", "axes"], ["out"])
        nodes = [axes, y]

    graph = helper.make_graph(
        nodes,
        "squeeze_test",
        inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(in_shape))],
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(out_shape))],
    )

    model = helper.make_model(graph, producer_name="squeeze_test")
    verify_with_ort(model, [in_shape], target=target, dev=dev, opset=opset)

verify_unsqueeze_with_neg_axes()
verify_unsqueeze_with_neg_axes(opset=13)

## `gather`

In [23]:
def verify_gather(in_shape, indices, axis, dtype):
    x = np.random.uniform(size=in_shape).astype(dtype)
    indices = np.array(indices, dtype="int64")
    out_np = np.take(x, indices, axis=axis)

    y = helper.make_node("Gather", ["in", "indices"], ["out"], axis=axis)

    graph = helper.make_graph(
        [y],
        "gather_test",
        inputs=[
            helper.make_tensor_value_info(
                "in", helper.np_dtype_to_tensor_dtype(np.dtype(dtype)), list(in_shape)
            ),
            helper.make_tensor_value_info("indices", TensorProto.INT64, list(indices.shape)),
        ],
        outputs=[
            helper.make_tensor_value_info(
                "out", helper.np_dtype_to_tensor_dtype(np.dtype(dtype)), list(out_np.shape)
            )
        ],
    )
    model = helper.make_model(graph, producer_name="gather_test")
    verify_with_ort_with_inputs(model, [x, indices], target=target, dev=dev, dtype=dtype)

verify_gather((4,), [1], 0, "int32")
verify_gather((1, 4), [0], 0, "int32")
verify_gather((4,), [[[1, 0], [0, 1]]], 0, "float32")
verify_gather((2, 2), [[[1, 0], [0, 1]]], 1, "int32")
verify_gather((3, 3, 3), [[[1, 0]]], -1, "int32")
verify_gather((4, 3, 5, 6), [[2, 1, 0, 0]], 0, "float32")

## `dynamic_gather`

In [26]:
from tvm import relay
dtype = "float32"
in_shape = [2, 2]
indices = 1
axis = 1
x = np.random.uniform(size=in_shape).astype(dtype)
indices = np.array(indices, dtype="int64")
out_np = np.take(x, indices, axis=axis)

indices = helper.make_node(
    "Constant",
    inputs=[],
    outputs=["indices"],
    value=onnx.helper.make_tensor(
        name="const_indices",
        data_type=onnx.TensorProto.INT64,
        dims=[],
        vals=[1],
    ),
)
y = helper.make_node("Gather", ["in", "indices"], ["out"], axis=axis)

graph = helper.make_graph(
    [indices, y],
    "gather_test",
    inputs=[
        helper.make_tensor_value_info(
            "in", helper.np_dtype_to_tensor_dtype(np.dtype(dtype)), ["?", "?"]
        ),
    ],
    outputs=[
        helper.make_tensor_value_info(
            "out", helper.np_dtype_to_tensor_dtype(np.dtype(dtype)), ["?"] * len(out_np.shape)
        )
    ],
)
model = helper.make_model(graph, producer_name="dynamic_gather_test")

mod, params = relay.frontend.from_onnx(model)

result = relay.create_executor("vm", mod=mod, device=dev, target=target).evaluate()(x, **params)
np.testing.assert_allclose(out_np, result.numpy(), rtol=1e-5, atol=1e-5)

## `gatherelements`

In [27]:
def verify_gatherelements(in_shape, indices, axis):
    x = np.random.uniform(size=in_shape).astype("float32")
    indices = np.array(indices, dtype="int32")

    y = helper.make_node("GatherElements", ["data", "indices"], ["output"], axis=axis)
    graph = helper.make_graph(
        [y],
        "gather_elements_test",
        inputs=[
            helper.make_tensor_value_info("data", TensorProto.FLOAT, list(in_shape)),
            helper.make_tensor_value_info("indices", TensorProto.INT32, list(indices.shape)),
        ],
        outputs=[helper.make_tensor_value_info("output", TensorProto.FLOAT, list(in_shape))],
    )
    model = helper.make_model(graph, producer_name="gather_elements_test")

    verify_with_ort_with_inputs(model, [x, indices], target=target, dev=dev)

verify_gatherelements((4,), [3, 0, 2, 1], 0)
verify_gatherelements((2, 2), [[1, 0], [0, 1]], 0)
verify_gatherelements((2, 2), [[0, 0], [1, 0]], 1)
verify_gatherelements((2, 2), [[1, 0], [0, 1]], 1)

indices = [
    [[1, 0, 0], [1, 0, 1], [0, 1, 1]],
    [[1, 1, 1], [1, 2, 1], [1, 0, 1]],
    [[1, 2, 1], [1, 2, 1], [1, 2, 1]],
]

verify_gatherelements((3, 3, 3), indices, 2)


## `scatter`

In [28]:
def verify_scatter(in_shape, indices, axis):
    x = np.random.uniform(size=in_shape).astype("float32")
    indices = np.array(indices, dtype="int32")
    updates = np.random.uniform(size=indices.shape).astype("float32")

    y = helper.make_node("Scatter", ["data", "indices", "updates"], ["output"], axis=axis)

    graph = helper.make_graph(
        [y],
        "scatter_test",
        inputs=[
            helper.make_tensor_value_info("data", TensorProto.FLOAT, list(in_shape)),
            helper.make_tensor_value_info("indices", TensorProto.INT32, list(indices.shape)),
            helper.make_tensor_value_info("updates", TensorProto.FLOAT, list(indices.shape)),
        ],
        outputs=[helper.make_tensor_value_info("output", TensorProto.FLOAT, list(in_shape))],
    )
    model = helper.make_model(graph, producer_name="scatter_test")
    # Scatter operator has been supported from version 9 and
    # deprecated since version 11 of the default ONNX operator set
    verify_with_ort_with_inputs(model, [x, indices, updates], target=target, dev=dev, opset=9)

verify_scatter((4,), [1], 0)
verify_scatter((1, 4), [[0]], 0)
verify_scatter((4,), [2, 3], 0)
verify_scatter((2, 2), [[1, 0], [0, 1]], 1)
verify_scatter((3, 3, 3), [[[-1, -3]]], -1)
verify_scatter((4, 3, 5, 6), [[[[2, 1, 0, 0]]]], 0)

## `scatter_elements`

In [29]:
def verify_scatter_elements(in_shape, indices, axis=0, reduction="update"):
    x = np.random.uniform(size=in_shape).astype("float32")
    indices = np.array(indices, dtype="int32")
    updates = np.random.uniform(size=indices.shape).astype("float32")

    scatter_elements_node = helper.make_node(
        "ScatterElements",
        ["data", "indices", "updates"],
        ["output"],
        axis=axis,
        reduction=reduction,
    )

    graph = helper.make_graph(
        [scatter_elements_node],
        "scatter_elements_test",
        inputs=[
            helper.make_tensor_value_info("data", TensorProto.FLOAT, list(in_shape)),
            helper.make_tensor_value_info("indices", TensorProto.INT32, list(indices.shape)),
            helper.make_tensor_value_info("updates", TensorProto.FLOAT, list(indices.shape)),
        ],
        outputs=[helper.make_tensor_value_info("output", TensorProto.FLOAT, list(in_shape))],
    )
    model = helper.make_model(graph, producer_name="scatter_elements_test")
    verify_with_ort_with_inputs(model, [x, indices, updates], target=target, dev=dev)

# Usual scatter for 1d input
verify_scatter_elements((4,), [2, 3])
# Usual scatter with specified positive axis
verify_scatter_elements((2, 2), [[1, 0], [0, 1]], 1)
# Usual scatter for 3d input with spicified negative indices and axis
verify_scatter_elements((3, 3, 3), [[[-1, -3]]], -1)
# Usual scatter for 4d input
verify_scatter_elements((4, 3, 5, 6), [[[[2, 1, 0, 0]]]])
# Scatter elements with addition reduction of duplicates
verify_scatter_elements(
    (3, 3, 3),
    [[[0, 2, 1], [1, 1, 1], [2, 1, 0]], [[0, 2, 1], [1, 1, 1], [2, 1, 0]]],
    0,
    "add",
)
# Scatter elements with reduction and specified axis
verify_scatter_elements((3, 3, 3), [[[2, 2, 2], [1, 1, 1], [0, 0, 0]]], 2, "add")
# Scatter elements with multiplication reduction of duplicates
verify_scatter_elements(
    (3, 3, 3),
    [[[0, 2, 1], [1, 1, 1], [2, 1, 0]], [[0, 2, 1], [1, 1, 1], [2, 1, 0]]],
    0,
    "mul",
)
# TODO(vvchernov): min and max options are supported from 18 version, but CI supports 17 only
# # Scatter elements with min reduction of duplicates
# verify_scatter_elements(
#     (3, 3, 3),
#     [[[0, 2, 1], [1, 1, 1], [2, 1, 0]], [[0, 2, 1], [1, 1, 1], [2, 1, 0]]],
#     0,
#     "min",
# )
# # Scatter elements with max reduction of duplicates
# verify_scatter_elements(
#     (3, 3, 3),
#     [[[0, 2, 1], [1, 1, 1], [2, 1, 0]], [[0, 2, 1], [1, 1, 1], [2, 1, 0]]],
#     0,
#     "max",
# )


## `slice`

In [30]:
def _test_slice_iteration_v1(indata, outdata, starts, ends, axes=None):
    if axes:
        y = helper.make_node("Slice", ["in"], ["out"], axes=axes, starts=starts, ends=ends)
    else:
        y = helper.make_node("Slice", ["in"], ["out"], starts=starts, ends=ends)

    graph = helper.make_graph(
        [y],
        "slice_test",
        inputs=[helper.make_tensor_value_info("in", TensorProto.FLOAT, list(indata.shape))],
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(outdata.shape))],
    )

    model = helper.make_model(graph, producer_name="slice_test")
    verify_with_ort_with_inputs(
        model, [indata], [outdata.shape], opset=1, target=target, dev=dev
    )

def _test_slice_iteration_v10(indata, outdata, **attrs):
    starts = attrs["starts"]
    ends = attrs["ends"]
    axes = None if "axes" not in attrs else attrs["axes"]
    steps = None if "steps" not in attrs else attrs["steps"]
    starts = np.asarray(starts)
    ends = np.asarray(ends)
    inputs = [
        helper.make_tensor_value_info("data", TensorProto.FLOAT, list(indata.shape)),
        helper.make_tensor_value_info("starts", TensorProto.INT64, list(starts.shape)),
        helper.make_tensor_value_info("ends", TensorProto.INT64, list(ends.shape)),
    ]
    initializer = [
        helper.make_tensor("starts", TensorProto.INT64, list(starts.shape), starts),
        helper.make_tensor("ends", TensorProto.INT64, list(ends.shape), ends),
    ]
    nodes = []

    if "add_noop_to_input_attrs" in attrs:

        def add_noop_to_input_attr(attr_name, attr):
            output_name = attr_name + "_output"

            ref_shape = list(np.array(attr).shape)
            ref_shape.insert(0, 1)
            ref_shape = tuple(ref_shape)
            ref_array = np.array(ref_shape)
            ref_node = onnx.helper.make_node(
                "Constant",
                inputs=[],
                outputs=["ref_in_" + attr_name],
                value=onnx.helper.make_tensor(
                    name="const_tensor__1_" + attr_name,
                    data_type=onnx.TensorProto.INT64,
                    dims=ref_array.shape,
                    vals=ref_array.flatten().astype(int),
                ),
            )
            in_shape = np.array(attr).shape
            in_array = np.array(in_shape)
            ref_node2 = onnx.helper.make_node(
                "Constant",
                inputs=[],
                outputs=["input_shape_" + attr_name],
                value=onnx.helper.make_tensor(
                    name="const_tensor__2_" + attr_name,
                    data_type=onnx.TensorProto.INT64,
                    dims=in_array.shape,
                    vals=in_array.flatten().astype(int),
                ),
            )

            reshape1_node = helper.make_node(
                "Reshape", [attr_name, "ref_in_" + attr_name], ["reshape_" + attr_name]
            )
            reshape2_node = helper.make_node(
                "Reshape", ["reshape_" + attr_name, "input_shape_" + attr_name], [output_name]
            )
            return [ref_node, ref_node2, reshape1_node, reshape2_node]

    slice_inputs = []
    for attr_name in ["starts", "ends", "axes", "steps"]:
        if attr_name not in attrs:
            continue
        if "add_noop_to_input_attrs" in attrs and attr_name in attrs["add_noop_to_input_attrs"]:
            nodes.extend(add_noop_to_input_attr(attr_name, attrs[attr_name]))
            slice_inputs.append(attr_name + "_output")
        else:
            slice_inputs.append(attr_name)

    if axes:
        axes = np.asarray(axes)
        inputs.append(
            helper.make_tensor_value_info("axes", TensorProto.INT64, list(axes.shape))
        )
        initializer.append(
            helper.make_tensor("axes", TensorProto.INT64, list(axes.shape), axes)
        )

    if steps:
        assert axes is not None and len(axes) == len(steps)
        steps = np.asarray(steps)
        inputs.append(
            helper.make_tensor_value_info("steps", TensorProto.INT64, list(axes.shape))
        )
        initializer.append(
            helper.make_tensor("steps", TensorProto.INT64, list(steps.shape), steps)
        )

    y = helper.make_node("Slice", ["data", *slice_inputs], ["out"])

    nodes.append(y)
    graph = helper.make_graph(
        nodes,
        "slice_test",
        inputs=inputs,
        outputs=[helper.make_tensor_value_info("out", TensorProto.FLOAT, list(outdata.shape))],
        initializer=initializer,
    )
    model = helper.make_model(graph, producer_name="slice_test")
    verify_with_ort_with_inputs(
        model, [indata], opset=10, freeze_params=True, use_vm=True, target=target, dev=dev
    )

x = np.random.randn(20, 10, 5).astype(np.float32)
_test_slice_iteration_v1(x, x[0:3, 0:10], starts=(0, 0), ends=(3, 10), axes=(0, 1))
_test_slice_iteration_v1(x, x[0:3, 0:10], starts=(0, 0), ends=(10, 3), axes=(1, 0))
_test_slice_iteration_v1(x, x[:, :, 3:4], starts=(0, 0, 3), ends=(20, 10, 4))
_test_slice_iteration_v1(x, x[:, 1:1000], starts=(1,), ends=(1000,), axes=(1,))
_test_slice_iteration_v1(x, x[:, 0:-1], starts=(0,), ends=(-1,), axes=(1,))
_test_slice_iteration_v10(x, x[0:3, 0:10], starts=(0, 0), ends=(3, 10), axes=(0, 1))
_test_slice_iteration_v10(x, x[0:3, 0:10], starts=(0, 0), ends=(10, 3), axes=(1, 0))
_test_slice_iteration_v10(x, x[:, :, 3:4], starts=(0, 0, 3), ends=(20, 10, 4))
_test_slice_iteration_v10(x, x[:, 1:1000], starts=(1,), ends=(1000,), axes=(1,))
_test_slice_iteration_v10(x, x[:, 0:-1], starts=(0,), ends=(-1,), axes=(1,))
_test_slice_iteration_v10(x, x[:, 0:-1], starts=(0,), ends=(-1,), axes=(-1,))
_test_slice_iteration_v10(
    x,
    x[0:3, 0:10],
    starts=(0, 0),
    ends=(3, 10),
    axes=(0, 1),
    add_noop_to_input_attrs=["starts"],
)
_test_slice_iteration_v10(
    x, x[:, :, 3:4], starts=(0, 0, 3), ends=(20, 10, 4), add_noop_to_input_attrs=["ends"]
)
_test_slice_iteration_v10(
    x, x[:, 1:1000], starts=(1,), ends=(1000,), axes=(1,), add_noop_to_input_attrs=["axes"]
)
_test_slice_iteration_v10(
    x,
    x[:, 0:-1],
    starts=(0,),
    ends=(-1,),
    axes=(1,),
    add_noop_to_input_attrs=["starts", "ends"],
)
_test_slice_iteration_v10(
    x,
    x[0:3, 0:10],
    starts=(0, 0),
    ends=(3, 10),
    axes=(0, 1),
    add_noop_to_input_attrs=["ends", "axes"],
)
_test_slice_iteration_v10(
    x,
    x[:, :, 3:4],
    starts=(0, 0, 3),
    ends=(20, 10, 4),
    add_noop_to_input_attrs=["starts", "axes"],
)
_test_slice_iteration_v10(
    x,
    x[:, 1:1000],
    starts=(1,),
    ends=(1000,),
    axes=(1,),
    add_noop_to_input_attrs=["starts", "ends", "axes"],
)
x = np.random.randn(1, 1, 1, 128).astype(np.float32)
_test_slice_iteration_v10(
    x, x, starts=(0, 0), ends=(9223372036854775807, 9223372036854775807), axes=(0, 3)
)

x = np.random.randn(4, 4).astype(np.float32)
_test_slice_iteration_v10(
    x, x[:, 1::2], starts=(1,), ends=(9223372036854775807,), axes=(1,), steps=(2,)
)
_test_slice_iteration_v10(
    x,
    x[0::1, 1::2],
    starts=(0, 1),
    ends=(4, 4),
    axes=(0, 1),
    steps=(1, 2),
)


2023-07-01 10:55:12.497323434 [W:onnxruntime:, model.cc:183 Model] ONNX Runtime only *guarantees* support for models stamped with opset version 7 or above for opset domain 'ai.onnx'. Please upgrade your model to opset 7 or higher. For now, this opset 1 model may run depending upon legacy support of some older opset version operators.
2023-07-01 10:55:12.497650501 [W:onnxruntime:, ort_transpose_optimizer.cc:24 ApplyImpl] Transpose optimizer failed: Unsupported ONNX opset: 1
2023-07-01 10:55:12.626164881 [W:onnxruntime:, model.cc:183 Model] ONNX Runtime only *guarantees* support for models stamped with opset version 7 or above for opset domain 'ai.onnx'. Please upgrade your model to opset 7 or higher. For now, this opset 1 model may run depending upon legacy support of some older opset version operators.
2023-07-01 10:55:12.626452322 [W:onnxruntime:, ort_transpose_optimizer.cc:24 ApplyImpl] Transpose optimizer failed: Unsupported ONNX opset: 1
2023-07-01 10:55:12.824904112 [W:onnxruntime

## `onnx_op_elementwise`